In [166]:
import json, openai, requests, warnings
warnings.filterwarnings("ignore")
import configparser
import uuid
from time import sleep
import tiktoken
import pandas as pd

In [2]:
credentials = configparser.ConfigParser()
credentials.read_file(open('apidata.config'))
my_key = credentials['OPENAI']['KEY']
my_org = credentials['OPENAI']['org']

In [157]:
def gpt3_question_collect(org, key,prompt,
                    engine='text-davinci-003',temp=0.01,
                   top_p=1.0,tokens=500,freq_pen=0.0,
                   pres_pen=0.0,stop=['asdfasdf','asdfasdf'],
                   filename='questions_file_rev01.jsonl'):
    """The function will take the prompt return the questions 
    that can be asked on the dataset. The same will be written 
    in questions_file.jsonl"""
    openai.api_key = key
    openai.organization = org
    max_retry=5
    retry = 0
    prompt = prompt.encode(encoding='ASCII',errors='ignore').decode()
    while True:
        try:
            response = openai.Completion.create(
            model=engine,
            prompt=prompt,
            temperature=temp,
            max_tokens=tokens,
            top_p=top_p,
            frequency_penalty=freq_pen,
            presence_penalty=pres_pen,
            stop=stop)
            
            text = response['choices'][0]['text'].strip()
            dict_reply = {'prompt':prompt,'questions':text}
            with open(filename,'a+') as taker:
                json.dump(dict_reply,fp=taker)
                taker.write('\n')
            
            return dict_reply
        except Exception as e:
            retry += 1
            if retry > max_retry:
                return f'GPT errored out {e}'
            sleep(1)

In [158]:
def gpt3_answer_collect(org, key,prompt_question,
                    engine='text-davinci-003',temp=0.01,
                   top_p=1.0,tokens=500,freq_pen=0.0,
                   pres_pen=0.0,stop=['asdfasdf','asdfasdf'],
                   filename='prompt_file_rev01.jsonl'):
    """The function will take the prompt return the questions 
    that can be asked on the dataset. The same will be written 
    in questions_file.jsonl"""
    openai.api_key = key
    openai.organization = org
    max_retry=5
    retry = 0
    prompt = prompt_question.encode(encoding='ASCII',errors='ignore').decode()
    while True:
        try:
            response = openai.Completion.create(
            model=engine,
            prompt=prompt_question,
            temperature=temp,
            max_tokens=tokens,
            top_p=top_p,
            frequency_penalty=freq_pen,
            presence_penalty=pres_pen,
            stop=stop)
            
            text = response['choices'][0]['text'].strip()
            dict_reply = {'prompt':prompt_question,
                          'completion':text}
            with open(filename,'a+') as taker:
                json.dump(dict_reply,fp=taker)
                taker.write('\n')
            
            return dict_reply
        except Exception as e:
            retry += 1
            if retry > max_retry:
                return f'GPT errored out {e}'
            sleep(1)

In [113]:
#reading in the dataset sample_data.csv

with open('space_titanic.csv','r') as sd:
    data = sd.readlines()

In [114]:
len(data)

8694

In [118]:
data[121]

'0128_01,Mars,False,D/3/S,TRAPPIST-1e,61.0,True,2353.0,334.0,9.0,316.0,2.0,Grohs Fles,False\n'

In [119]:
data[61]

'0066_01,Earth,False,G/6/P,TRAPPIST-1e,62.0,False,1.0,153.0,197.0,0.0,460.0,Diandy Pecketton,False\n'

In [120]:
data[181]

'0202_01,Europa,True,B/9/P,55 Cancri e,45.0,False,0.0,0.0,0.0,0.0,0.0,Pomera Embleng,True\n'

In [121]:
data[1:5]

['0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False\n',
 '0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True\n',
 '0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False\n',
 '0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False\n']

In [122]:
data[5:10]

['0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True\n',
 '0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True\n',
 '0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,True\n',
 '0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,,Candra Jacostaffey,True\n',
 '0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True\n']

In [123]:
f'{data[0]}+{data[1:5]}'

"PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported\n+['0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False\\n', '0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True\\n', '0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False\\n', '0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False\\n']"

In [124]:
f'{data[0]}+{data[5:10]}'

"PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported\n+['0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True\\n', '0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True\\n', '0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,True\\n', '0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,,Candra Jacostaffey,True\\n', '0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True\\n']"

In [9]:
# To get the tokeniser corresponding to a specific model in the OpenAI API:
enc = tiktoken.encoding_for_model("text-davinci-003")

In [10]:
enc.encode("This is my first use of tiktoken")

[1212, 318, 616, 717, 779, 286, 256, 1134, 30001]

In [11]:
enc.decode(enc.encode("This is my first use of tiktoken"))

'This is my first use of tiktoken'

In [12]:
enc.n_vocab

50281

In [13]:
def num_tokens_from_string(string: str, 
                           encoding_name: str = 'text-davinci-003') -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [14]:
num_tokens_from_string(string="This is the first use of tiktoken",
                       encoding_name='text-davinci-003')

9

In [15]:
### prompt design

prompt = f"""
[Dataset] = {data[0]}+{data[1:60]} 
prompt:Based on above [Dataset] list 10 questions that I can ask

Answer:
"""

In [16]:
num_tokens_from_string(prompt)

3527

### The prompt and the answer should be total 4001 token

In [17]:
4001 - num_tokens_from_string(prompt)

474

Psuedocode:

1) Split the dataset into small chunks so it can fit inside the prompt. 

1a) Each dataset should be given a unique name

2) Use the dataset chunks and send the prompt asking for the 15 questions that you can ask

3) Collect the questions and attach it with the dataset chunk and write in a seperate file

Do this for 15 chunks of the dataset you will have 225 questions

4) Each dataset chunks will become 15 prompts with individual questions

Prompt: Refer the {Dataset(num)} that was provided to you. {question}

Answer:

5) The prompt, along with the dataset, dataset num, question is considered as one prompt. The answer the model provides is considered as completion. 

{"prompt":dataset{num}=dataset_values,Refer the {dataset(num)} that was provided to you. {question}, "completion":Answer}

6) Train the model...

In [18]:
#We can train with 144 datasets
len(data)/60

144.9

In [125]:
data_chunks = []
rng1 = 1

In [126]:
data_chunks

[]

In [128]:
for x in list(range(1,5)):
    print(x)

1
2
3
4


In [129]:
for x in list(range(1,3)):
    rng2 = rng1 + 5
    print(f'before spliting data {rng1} and {rng2}')
    data_t = f'{data[0]}+{data[rng1:rng2]}'
    print(data_t)
    prompt_made = f"""
    dataset_{x}={data_t}
    
    Prompt:Refer to the [dataset_{x}] and List 25 questions that can be asked from it
    
    Answer:
    """
    data_chunks.append(prompt_made)
    print(f'Number of tokens for the dataset {x + 1} is {num_tokens_from_string(prompt_made)}')
    rng1 = rng1 + 5

before spliting data 1 and 6
PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
+['0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False\n', '0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True\n', '0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False\n', '0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False\n', '0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True\n']
Number of tokens for the dataset 2 is 384
before spliting data 6 and 11
PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
+['0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True\n', '0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0

In [142]:
data_chunks[0]

"\n    dataset_1=PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported\n+['0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False\\n', '0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True\\n', '0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False\\n', '0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False\\n', '0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True\\n']\n    \n    Prompt:Refer to the [dataset_1] and List 25 questions that can be asked from it\n    \n    Answer:\n    "

In [143]:
chunk0_df = chunk_to_df(1,data_chunks[0])
chunk0_df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [144]:
chunk0_df.groupby('Destination').count()

,PassengerId,HomePlanet,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
Destination,,,,,,,,,,,,,
TRAPPIST-1e,5,5,5,5,5,5,5,5,5,5,5,5,5


### Initial the execution was with temp at 1.0

In [159]:
question_1 = gpt3_question_collect(org=my_org,key=my_key,
                      prompt=data_chunks[0])

In [160]:
question_1

{'prompt': "\n    dataset_1=PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported\n+['0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False\\n', '0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True\\n', '0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False\\n', '0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False\\n', '0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True\\n']\n    \n    Prompt:Refer to the [dataset_1] and List 25 questions that can be asked from it\n    \n    Answer:\n    ",
 'questions': '1. What is the average age of passengers in the dataset?\n2. How many passengers have CryoSleep enabled?\n3. How many passengers have VIP status?\n4. How many passengers have RoomService enabled?\n5. What

In [163]:
gpt3_answer_collect(org=my_org,key=my_key,
                   prompt_question=f"""
dataset_1={data[0]}+{data[1:5]}

Prompt:Refer to the [dataset_1] and answer How many rows are there in the dataset?

Answer:""")

{'prompt': "\ndataset_1=PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported\n+['0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False\\n', '0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True\\n', '0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False\\n', '0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False\\n']\n\nPrompt:Refer to the [dataset_1] and answer How many rows are there in the dataset?\n\nAnswer:",
 'completion': 'There are four rows in the dataset.'}

In [164]:
gpt3_answer_collect(org=my_org,key=my_key,
                   prompt_question=f"""
dataset_1={data[0]}+{data[1:5]}

Prompt:Refer to the [dataset_1] and answer Which is the most travelled Destination?

Answer:""")

{'prompt': "\ndataset_1=PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported\n+['0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False\\n', '0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True\\n', '0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False\\n', '0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False\\n']\n\nPrompt:Refer to the [dataset_1] and answer Which is the most travelled Destination?\n\nAnswer:",
 'completion': 'TRAPPIST-1e'}

In [165]:
gpt3_answer_collect(org=my_org,key=my_key,
                   prompt_question=f"""
dataset_1={data[0]}+{data[1:5]}

Prompt:Refer to the [dataset_1] and answer What is the average Spa spending of passengers?

Answer:""")

{'prompt': "\ndataset_1=PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported\n+['0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False\\n', '0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True\\n', '0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False\\n', '0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False\\n']\n\nPrompt:Refer to the [dataset_1] and answer What is the average Spa spending of passengers?\n\nAnswer:",
 'completion': 'The average Spa spending of passengers in the dataset_1 is 2,817.'}

In [181]:
gpt3_answer_collect(org=my_org,key=my_key,
                   prompt_question=f"""
dataset_1={data[0]}+{data[1:5]}

Prompt:Refer to the [dataset_1] and list all the Names in the dataset?

Answer:""")

{'prompt': "\ndataset_1=PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported\n+['0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False\\n', '0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True\\n', '0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False\\n', '0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False\\n']\n\nPrompt:Refer to the [dataset_1] and list all the Names in the dataset?\n\nAnswer:",
 'completion': 'Maham Ofracculy, Juanna Vines, Altark Susent, Solam Susent'}

In [182]:
gpt3_answer_collect(org=my_org,key=my_key,
                   prompt_question=f"""
Prompt:Refer to the [dataset_1] and tell me which passenger spent more on Spa?

Answer:""")

{'prompt': '\nPrompt:Refer to the [dataset_1] and tell me which passenger spent more on Spa?\n\nAnswer:',
 'completion': 'According to the dataset_1, the passenger who spent the most on Spa was Passenger ID 8, who spent $200.'}

In [183]:
gpt3_answer_collect(org=my_org,key=my_key,
                   prompt_question=f"""
Prompt:Refer to the dataset_1 that is in your memory and tell me which passenger spent more on Spa?

Answer:""")

{'prompt': '\nPrompt:Refer to the dataset_1 that is in your memory and tell me which passenger spent more on Spa?\n\nAnswer:',
 'completion': 'The passenger who spent more on Spa from the dataset_1 is Passenger ID 8.'}

In [184]:
gpt3_answer_collect(org=my_org,key=my_key,
                   prompt_question=f"""
dataset_1={data[0]}+{data[1:5]}. Remember [dataset_1] for long time.

Prompt:Refer to the [dataset_1] and list all the Names in the dataset?

Answer:""")

{'prompt': "\ndataset_1=PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported\n+['0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False\\n', '0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True\\n', '0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False\\n', '0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False\\n']. Remember [dataset_1] for long time.\n\nPrompt:Refer to the [dataset_1] and list all the Names in the dataset?\n\nAnswer:",
 'completion': 'Maham Ofracculy, Juanna Vines, Altark Susent, Solam Susent'}

In [185]:
gpt3_answer_collect(org=my_org,key=my_key,
                   prompt_question=f"""
Prompt:Refer to the dataset_1 that is in your memory and list all the passenger names?

Answer:""")

{'prompt': '\nPrompt:Refer to the dataset_1 that is in your memory and list all the passenger names?\n\nAnswer:',
 'completion': 'John Smith, Jane Doe, Sarah Johnson, Mark Williams, Emma Brown, David Jones, Michael Miller, Jessica Taylor, Ryan Davis, Ashley Anderson'}

In [186]:
gpt3_answer_collect(org=my_org,key=my_key,
                   prompt_question=f"""
Prompt:Refer to the [dataset_1] that is in your memory and list all the passenger names?

Answer:""")

{'prompt': '\nPrompt:Refer to the [dataset_1] that is in your memory and list all the passenger names?\n\nAnswer:',
 'completion': '1. John Smith\n2. Jane Doe\n3. Robert Johnson\n4. Sarah Williams\n5. Mark Brown\n6. Jessica Miller\n7. David Jones\n8. Emily Davis\n9. Andrew Taylor\n10. Samantha Wilson'}

In [188]:
chunk_2_list = data_chunks[1].split('\\n')
chunk_2_list

["\n    dataset_2=PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported\n+['0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True",
 "', '0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,True",
 "', '0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,,Candra Jacostaffey,True",
 "', '0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True",
 "', '0008_01,Europa,True,B/1/P,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,Erraiam Flatic,True",
 "']\n    \n    Prompt:Refer to the [dataset_2] and List 25 questions that can be asked from it\n    \n    Answer:\n    "]

In [191]:
clean_step1 = chunk_2_list[0].strip(' \n').replace('=','').replace('dataset_2','')
clean_step2 = clean_step1.split('\n+[')
header = clean_step2[0]
row1 = clean_step2[1]

In [192]:
header

'PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported'

In [193]:
row1

"'0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True"

In [194]:
clean_row = []
for dat in chunk_2_list[1:]:
    clean_row.append(dat.strip(" ',"))

In [195]:
clean_row[1]

'0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,,Candra Jacostaffey,True'

In [196]:
clean_row.insert(0,row1.strip("'")) 

In [197]:
clean_row.insert(0,header)

In [198]:
clean_data_list = [data.split(',') for data in clean_row]

In [200]:
pd.DataFrame(clean_data_list[1:-1],columns=clean_data_list[0])

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True
1,0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,True
2,0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,,Candra Jacostaffey,True
3,0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True
4,0008_01,Europa,True,B/1/P,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,Erraiam Flatic,True


In [137]:
def chunk_to_df(chunk_num,data_chunk):
    
    start = data_chunk.split('\\n')
    
    clean_step1 = start[0].strip(' \n').replace('=','').replace(f'dataset_{chunk_num}','')

    clean_step2 = clean_step1.split('\n+[')
    
    header = clean_step2[0]
    row1 = clean_step2[1]
    
    clean_row = []
    for dat in start[1:]:
        clean_row.append(dat.strip(" ',"))
        
    clean_row.insert(0,row1.strip("'"))
    clean_row.insert(0,header)
    
    clean_data_list = [data.split(',') for data in clean_row]
    
    chunk_df = pd.DataFrame(clean_data_list[1:-1],columns=clean_data_list[0])
    
    return chunk_df

In [203]:
with open('questions_file_rev01.jsonl','r') as qpf:
    json_data = json.load(qpf)

In [205]:
def question_extractor(reply_dict):
    extracted = []
    quest_list = reply_dict['questions'].split('\n')
    
    for que in quest_list:
        ready = que.split('.')[1].strip(' ')
        extracted.append(ready)
    
    return extracted

In [76]:
question = questionList[0].split('.')[1].strip(' ')

In [207]:
quest_set0 = question_extractor(json_data[0])

In [214]:
len(quest_set0)

25

In [206]:
quest_set1 = question_extractor(json_data[1])

In [212]:
#make the prompts with the quest sets
def prompt_question(num, question, data,rng1, rng2):
    data_t = f'''{data[0]}+{data[rng1:rng2]}'''
    buildQuestion = f"""
dataset_{num}={data_t}
    
Prompt:Refer to the [dataset_{num}] and answer {question} 

Answer:
"""
    return prompt_made

In [213]:
prompt_question(1,quest_set0[0],data=data,rng1=1,rng2=5)

"\ndataset_1=PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported\n+['0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False\\n', '0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True\\n', '0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False\\n', '0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False\\n']\n    \nPrompt:Refer to the [dataset_1] and answer What is the age range of the passengers in dataset_1? \n\nAnswer:\n"

In [215]:
#Iterate over the question sets
for ind,que in enumerate(quest_set0[1:5]):
    quest_ref_dataset = prompt_question(1,que,data=data,rng1=1,rng2=5)
    gpt3_answer_collect(key=my_key,org=my_org,
                        prompt_question=quest_ref_dataset)

In [217]:
chunk_to_df(1,data_chunks[0])

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Hypothesis I want to check is:
After giving the datasets as prompt for 25 question and then asking the model to refer the memory dataset and answer

In [219]:
question_2 = gpt3_question_collect(org=my_org,key=my_key,
                      prompt=data_chunks[1])

In [220]:
question_2

{'prompt': "\n    dataset_2=PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported\n+['0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True\\n', '0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,True\\n', '0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,,Candra Jacostaffey,True\\n', '0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True\\n', '0008_01,Europa,True,B/1/P,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,Erraiam Flatic,True\\n']\n    \n    Prompt:Refer to the [dataset_2] and List 25 questions that can be asked from it\n    \n    Answer:\n    ",
 'questions': '1. What is the average age of passengers in the dataset?\n2. How many passengers have CryoSleep enabled?\n3. How many passengers have VIP status?\n4. How many passengers are travelling to PSO J318.5-22?\n5. How many pa

In [223]:
type(question_2)

dict

In [224]:
quest_set1 = question_extractor(question_2)

In [218]:
#Iterate over the question sets
for ind,que in enumerate(quest_set0[5:]):
    quest_ref_dataset = prompt_question(1,que,data=data,rng1=1,rng2=5)
    gpt3_answer_collect(key=my_key,org=my_org,
                        prompt_question=quest_ref_dataset)

In [215]:
#Iterate over the question sets
for ind,que in enumerate(quest_set0[1:5]):
    quest_ref_dataset = prompt_question(1,que,data=data,rng1=1,rng2=5)
    gpt3_answer_collect(key=my_key,org=my_org,
                        prompt_question=quest_ref_dataset)

In [226]:
#Iterate over the question sets
for ind,que in enumerate(quest_set0[6:]):
    quest_ref_dataset = prompt_question(1,que,data=data,rng1=1,rng2=5)
    gpt3_answer_collect(key=my_key,org=my_org,
                        prompt_question=quest_ref_dataset)

In [225]:
#Iterate over the question sets
for ind,que in enumerate(quest_set1):
    quest_ref_dataset = prompt_question(2,que,data=data,rng1=6,rng2=11)
    gpt3_answer_collect(key=my_key,org=my_org,
                        prompt_question=quest_ref_dataset)

In [227]:
def file_upload(filename, purpose='fine-tune'):
    resp = openai.File.create(purpose=purpose,file=open(file=filename,
                                                        mode='rb'))
    print(resp)
    return resp

def finetune_model(fileId, suffix,model='davinci',api_key=my_key):
    header = {'Content-Type':'application/json',
              'Authorization':f'Bearer {api_key}'}
    payload = {'training_file':fileId,'model':model,'suffix':suffix}
    resp = requests.request(method='POST',
                            url='https://api.openai.com/v1/fine-tunes',
                           headers=header,
                           json=payload,timeout=40)
    print(resp.json())
    
def finetune_get(ftId,api_key=my_key):
    header = {'Content-Type':'application/json',
              'Authorization':f'Bearer {api_key}'}
    resp = requests.request(method='GET',
                url=f'https://api.openai.com/v1/fine-tunes/{ftId}',
                           headers=header,timeout=40)
    print(resp.json())
    
def finetune_event(ftId,api_key=my_key):
    header = {'Content-Type':'application/json',
              'Authorization':f'Bearer {api_key}'}
    resp = requests.request(method='GET',
                url=f'https://api.openai.com/v1/fine-tunes/{ftId}/events',
                           headers=header,timeout=40)
    print(resp.json())

In [228]:
file_upload(filename='prompt_file_rev01.jsonl')

{
  "bytes": 56412,
  "created_at": 1678416209,
  "filename": "file",
  "id": "file-nmGcIdqQGgI66E2GQiTZHddJ",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}


<File file id=file-nmGcIdqQGgI66E2GQiTZHddJ at 0x7f08d648fe70> JSON: {
  "bytes": 56412,
  "created_at": 1678416209,
  "filename": "file",
  "id": "file-nmGcIdqQGgI66E2GQiTZHddJ",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}

In [229]:
finetune_model(fileId='file-nmGcIdqQGgI66E2GQiTZHddJ',
               api_key=my_key,suffix='space_titanic_r03',
              model='davinci')

{'object': 'fine-tune', 'id': 'ft-AnItXe2ZVM8ojxfrTA4QUyDK', 'hyperparams': {'n_epochs': 4, 'batch_size': None, 'prompt_loss_weight': 0.01, 'learning_rate_multiplier': None}, 'organization_id': 'org-QWWIpNe4Z3KsRVI0YR6lhyJW', 'model': 'davinci', 'training_files': [{'object': 'file', 'id': 'file-nmGcIdqQGgI66E2GQiTZHddJ', 'purpose': 'fine-tune', 'filename': 'file', 'bytes': 56412, 'created_at': 1678416209, 'status': 'processed', 'status_details': None}], 'validation_files': [], 'result_files': [], 'created_at': 1678416227, 'updated_at': 1678416227, 'status': 'pending', 'fine_tuned_model': None, 'events': [{'object': 'fine-tune-event', 'level': 'info', 'message': 'Created fine-tune: ft-AnItXe2ZVM8ojxfrTA4QUyDK', 'created_at': 1678416227}]}


In [230]:
finetune_event(ftId='ft-AnItXe2ZVM8ojxfrTA4QUyDK',
               api_key=my_key)

{'object': 'list', 'data': [{'object': 'fine-tune-event', 'level': 'info', 'message': 'Created fine-tune: ft-AnItXe2ZVM8ojxfrTA4QUyDK', 'created_at': 1678416227}]}


In [231]:
finetune_get(ftId='ft-gOeTtRL8FbQYMKzrheNNBm8H',
               api_key=my_key)

{'object': 'fine-tune', 'id': 'ft-gOeTtRL8FbQYMKzrheNNBm8H', 'hyperparams': {'n_epochs': 4, 'batch_size': 1, 'prompt_loss_weight': 0.01, 'learning_rate_multiplier': 0.1}, 'organization_id': 'org-QWWIpNe4Z3KsRVI0YR6lhyJW', 'model': 'davinci', 'training_files': [{'object': 'file', 'id': 'file-B7AJ8LI7onSaVqC3ZaW68NR9', 'purpose': 'fine-tune', 'filename': 'file', 'bytes': 10818, 'created_at': 1678374670, 'status': 'processed', 'status_details': None}], 'validation_files': [], 'result_files': [{'object': 'file', 'id': 'file-rDCVaQOPP5fTeuqFhUiisDfo', 'purpose': 'fine-tune-results', 'filename': 'compiled_results.csv', 'bytes': 10717, 'created_at': 1678386998, 'status': 'processed', 'status_details': None}], 'created_at': 1678377767, 'updated_at': 1678386998, 'status': 'succeeded', 'fine_tuned_model': 'davinci:ft-kamal:space-titanic-r02-2023-03-09-18-36-37', 'events': [{'object': 'fine-tune-event', 'level': 'info', 'message': 'Created fine-tune: ft-gOeTtRL8FbQYMKzrheNNBm8H', 'created_at': 16